# Analiza wariantów

In [ ]:
import os                               # moduł OS języka Python
user_name = os.environ.get('USER')      # pobieramy zmienną środowiskową USER
bucket = f"gs://edugen-lab-{user_name}" # konstruujemy sciezke dostepowa do pliku
print(bucket)

In [ ]:
# sprawdzenie czy dane są dostępne
! gsutil ls -r $bucket


## Przygotowanie sesji Spark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
.builder \
.config('spark.driver.memory','1g') \
.config('spark.executor.memory', '2g') \
.getOrCreate()

## Stworzenie tabeli z wariantami

In [ ]:
var_path=f"{bucket}/vcf/mother10.vcf"
table_var = 'variants'

spark.sql(f'DROP TABLE IF EXISTS {table_var}')

spark.sql(f'CREATE TABLE IF NOT EXISTS {table_var} \
USING org.biodatageeks.sequila.datasources.VCF.VCFDataSource \
OPTIONS(path "{var_path}")')

## Weryfikacja danych

<div class="alert alert-block alert-warning">
<b>Zadanie 4_1:</b> 
Napisz polecenie, które pokaże strukturę (kolumny i ich typy) tabeli z wariantami
    </div>

In [ ]:
spark.sql(f'select count (*) from {table_var}').show()

Widzimy, że ALT jest rodzaju tablicowego, sprawdźmy, czy są warianty mutlialleliczne?

In [ ]:
spark.sql(f'select * from {table_var} where size(alt) > 1').show()

In [ ]:
spark.sql(f'select * from {table_var} where size(alt) > 1').count() # ile ich jest ? 

In [ ]:
spark.sql(f'select genotypes from {table_var}').show(truncate=False) # tabela w postaci nieznormalizowanej -> mamy listę wartości

Mamy warianty multialleliczne. Zeby rozbic je na warianty bi-alleliczne uzyjemy narzedzia VT uruchamianego na kilku executorach. Importujemy potrzebny moduł seqtender 

In [ ]:
from pyseqtender import SeqTenderAnnotation
seq_anno = SeqTenderAnnotation(spark)
dec_variants = seq_anno.pipe_variants (var_path, "vt decompose -")

In [ ]:
# sprawdzimy liczność wyniku
dec_variants.count()

In [ ]:
# zapiszemy wyniki w kubełku
var_dec_path = f"{bucket}/vcf/mother_dec.vcf"
seq_anno.save_variants (var_dec_path, dec_variants)

In [ ]:
table_var_dec = 'variants_dec'
spark.sql(f'DROP TABLE IF EXISTS {table_var_dec}')
spark.sql(f'CREATE TABLE IF NOT EXISTS {table_var_dec} \
USING org.biodatageeks.sequila.datasources.VCF.VCFDataSource \
OPTIONS(path "{var_path}")')


In [ ]:
spark.sql(f"select * from {table_var_dec}").show()

Zrealizujemy jeszcze proces adnotacji wariantów

In [ ]:
from pyseqtender import SeqTenderAnnotation
seq_anno = SeqTenderAnnotation(spark)

vcf_path=f"{bucket}/vcf/mother_dec.vcf"

cache_dir = "/mnt/data/annotation/102.0"
vep_version="102"
annotate_cmd = f"""vep --dir {cache_dir} --pick_allele --format vcf --no_stats --force_overwrite  --uniprot  -cache --vcf -offline --assembly GRCh37
        -o stdout """.replace("\n   ", "")

annotated = seq_anno.pipe_variants(vcf_path, annotate_cmd)

In [ ]:
annotated.count()

In [ ]:
var_anno_path = f"{bucket}/vcf/mother_anno.vcf"
seq_anno.save_variants (var_anno_path, annotated)

In [ ]:
var_anno_path=f"{bucket}/vcf/mother_anno.vcf"
table_var_anno= 'variants_anno'
spark.sql(f'DROP TABLE IF EXISTS {table_var_anno}')
spark.sql(f'CREATE TABLE IF NOT EXISTS {table_var_anno} \
USING org.biodatageeks.sequila.datasources.VCF.VCFDataSource \
OPTIONS(path "{var_anno_path}")')

In [ ]:
anno = spark.sql(f"select * from {table_var_anno}")

<div class="alert alert-block alert-warning">
<b>Zadanie 4_2:</b> 
Wyświetl schemat DF anno i podaj liczbę kolumn. Jakiego typu są nowe kolumny?
    </div>

In [ ]:
from  pyspark.sql.functions import *
anno1 = anno.withColumn("INFO_CSQ2", explode(("INFO_CSQ")))
anno2 = anno1.selectExpr( "*", " INFO_CSQ2.*")

<div class="alert alert-block alert-warning">
<b>Zadanie 4_3:</b> 
Pogrupuj warianty po kolumnie IMPACT oraz oblicz ile jest wariantów w każdej grupie. Zrób to samo dla kolumny Consequence oraz SYMBOL.
    </div>


## Biblioteka Pandas

https://pandas.pydata.org/

Moduł Pandas jest biblioteką Pythonową do manipulacji danymi. W szczegolnosci w pandas mozemy stworzyc ramki danych i wykonywac na niej analize, agregacje oraz wizualizacje danych. 
Przy nieduzych zbiorach danych i prostych operacjach to doskonała biblioteka. Jednak kiedy zbior danych sie rozrasta lub kiedy wymagane sa zlozone transformacje to operacje moga byc wolne.

Operacje na rozproszonych danych sa szybsze. Ale tu takze napotykamy ograniczenia np trudność w wizualizacji danych.

In [ ]:
spark.sql(f"select * from {table_var_anno} LIMIT 10").toPandas()

**Ważne** Metoda toPandas() na ramce pyspark, konwertuje ramkę pyspark do ramki pandas. Wykonuje akcje pobrania wszystkich danych z executorów (z JVM) i transfer do  programu sterujacego (driver) i konwersje do typu Pythonowego w notatniku. Ze względu na ograniczenia pamięciowe w programie sterującym należy to wykonywać na podzbiorach danych.

In [ ]:
anno2.toPandas()

Nie pokazują się wszystkie kolumny (w środku mamy ...). Aby wyświetlić całość musimy ustawić:

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [ ]:
anno_pandas = anno2.toPandas()
anno_pandas

# Matplotlib
https://pandas.pydata.org/pandas-docs/version/0.23.4/api.html#api-dataframe-plotting

In [ ]:
import matplotlib.pyplot as plt
anno2.groupby("ref").count().toPandas().plot.bar(x='ref')

In [ ]:
anno2.groupby("ref").count().toPandas().plot.pie(y='count')

<div class="alert alert-block alert-warning">
<b>Zadanie 4_4:</b> 
Przerób powyższe dwa rysunki, tak żeby pokazywały rozkład referencji tylko dla SNV (bez indeli).
    </div>



<div class="alert alert-block alert-warning">
<b>Zadanie 4_5:</b> 
Przygotuj pie chart dla kolumny Consequences. Uwzględnij jedną konsekwencję dla każdego wariantu.
    </div>



In [ ]:
spark.stop()